## Final Assessment Programming 1
By: Jacob Menzinga (357758)

##### Introduction
In the united states the crime rates steadily increased up untill the 1990's. Then, a sharp decrease in crime rates was observed. To explain this decrease, researchers have come up with multple reasons. One factor that drew attention was lead exposure. This lead to the Lead-Crime hypothesis. A lot of papers since have been published on this. One example is a meta analysis published by Anthony Higney, et al. (2022) in wich they write: "Our estimates suggest the abatement of lead pollution may be responsible for 7–28% of the fall in homicide in the US"

For the final asessment of programming 1 I decided to investigate if such a relation is also visible in the netherlands. To take blood-lead values of the whole population of the Netherlands is a bit unethical and expensive so I decided to look at the amount of lead in  wastewater instead

##### Hypotheses
A higher amount of lead in wastewater correlates to a higher incidence of violent crimes

##### Data sources
1. <a href="https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=7477&_theme=309">Wastewater treatment in the netherlands</a><br>
From this dataset the following features were selected:

    - Onderwerpen -> Aanvoer van afvalwater -> Hoeveelheden:
        - Volume afvalwater in 1000 m3
        - Lood in kg

    - Regios:
        - Provincies

    - Perioden:
        - from 2010 up to and including 2020

2. <a href="https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=83648NED&_theme=406">Registered crime in the netherlands</a><br>
From this dataset the following features were selected:

    - Onderwerpen -> Geregistreerde misdrijven:
        - Geregistreerde misdrijven per 1000 inw.

    - Soort misdrijf:
        - 111 Diefstal en inbraak met geweld
        - 15 Afpersing en afdreiging
        - 21 Vernieling en beschadiging
        - 221 Openlijke geweldpleging
        - 23 Brandstichting / ontploffing
        - 3 Gewelds- en seksuele misdrijven
        - 7 Vuurwapenmisdrijven

    - Regios:
        - Provincies

    - Perioden:
        - from 2010 up to and including 2020
    


##### Reading in the data

In [ ]:
# Imports
import yaml

import pandas as pd
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show
output_notebook()

import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

import folium
from folium import plugins

import DS_Module as ds

##### Supporting Functions

In [ ]:
def check_data(df):
    """
    A function to check any dataframe for:
        - Missing data
        - Datatypes
        - Descriptive statistics
        
    It then prints it's findings 

    Args:
        df (pd.DataFrame): Any dataframe.
    """
    missing_data = df.isna().sum()
    if missing_data.values.sum() == 0:
        print('Missing data:')
        print('No missing data :)')
        missing_loc = 'None'
    
    else:
        # missing_data['perc. of data'] = df.isna().sum()/(len(df))*100
        missing_loc = df[df.isnull().any(axis=1)]
        print(f"Missing data per column:\n{missing_data}\n")
        print("The missing data is located in the following rows:")
        print(missing_loc)
    
    
    dtypes = df.dtypes
    print('\nData types:')
    print(dtypes)
    
    describe = df.describe()
    print(f'\nDescription of the dataframe')
    print(describe)
    

##### Importing and cleaning data

In [ ]:
with open('config.yaml') as stream:
    config = yaml.safe_load(stream)
    
crime_df = pd.read_csv(config['crime'], delimiter=';')
lead_df = pd.read_csv(config['lead'], delimiter=';')

First I'll have a look at crime_df

In [ ]:
crime_df.rename(columns= {'SoortMisdrijf':'Crime',
                         'RegioS':'Provence', 'Perioden':'Year',
                         'GeregistreerdeMisdrijvenPer1000Inw_3':'Incidence'},
                inplace=True)
crime_df

In [ ]:
check_data(crime_df)

Two things I took away from the datacheck:
1) There are a lot of missing values in the PV99 region. I looked this region code up in the metadata file of the crime dataset (also downloadable from the above link) and this is a category for 'uncatogarisable' data so I will drop these rows.

2) I want to turn the Year and Incidence columns into int and float dtypes respectively

In [ ]:
# Dropping the PV99 region
crime_df = crime_df[crime_df['Provence'] != 'PV99  ']
crime_df
# This seems to have messed up the index a bit, 
# concidering there's 924 rows in the df but the index gows up to 989.

In [ ]:
crime_df = crime_df.reset_index().drop('index', axis=1)

In [ ]:
# Checking the values in the Incidence and Year columns
print(crime_df['Incidence'].unique())
print(crime_df['Year'].unique())

I'm assuming the '       .' values for incidence should be 0, to check this I want to plot the 5 datapoints before these values to see if they show a trend decreasing toward 0

In [ ]:

missing_crime = crime_df[crime_df['Incidence']=='       .'].index
missing_crime = np.array(missing_crime)

trend_ind = np.empty(0)

for i in range (6):
    trend_ind = np.append(trend_ind, missing_crime-i)
    
trend_ind = np.unique(trend_ind)

# this list I will use after after replacing the '.' values to 0 to check if 
# my assumption was correct

In [ ]:
# replacing the '       .' value with 0
crime_df['Incidence'] = crime_df['Incidence'].str.replace('       .', '0', regex=False)

# Typecasting the Year and Incidence columns
crime_df['Year'] = crime_df['Year'].str.replace('JJ00','').astype(int)
crime_df['Incidence'] = crime_df['Incidence'].astype(float)

In [ ]:
# Plotting the incidence, per region and crime. 0 values are interpolated.
hv.output(widget_location='top_left')
crime_df.iloc[trend_ind
             ].sort_values(by='Year').hvplot.line(x='Year', 
                                                  y='Incidence', 
                                                  groupby=['Crime', 'Provence'])

In [ ]:
# Now that that's done, I'll run the check data again to see if I got rid of all the missing data
check_data(crime_df)

The crime_df has the different types of crime in one column, I would like each crime as a different feature with he incedence as their value

In [ ]:
crime_df = crime_df.set_index(['Provence','Year']).pivot(columns='Crime', values='Incidence').reset_index()
crime_df

And also a total incidence would be usefull

In [ ]:
crime_df['Total_incidence'] = crime_df[[
    'CRI1110', 'CRI1500', 'CRI2100', 'CRI2210', 'CRI2300', 'CRI3000', 'CRI7000']
                                       ].sum(axis=1)
crime_df

Now its time for lead_df

In [ ]:
lead_df.rename(columns={'RegioS':'Provence', 'Perioden':'Year',
                        'VolumeAfvalwater_43':'Vol_Wastewater', 
                        'Lood_52':'Lead'}, inplace= True)
lead_df

In [ ]:
# First some general information
check_data(lead_df)

In this dataframe I want to change the Year and Lead columns to intergers

In [ ]:
print(lead_df['Lead'].unique())

In this dataset there's also '       .' values.
A value of 0kg lead in the wastewater does not make sense to me given all the other values are at least above 100, and the vol_wastewater is not decreased in these rows. 

I'm going to interpolate these values and plot them between non interpolated values to see if this makes sense

In [ ]:
missing_lead = lead_df[lead_df['Lead']=='       .'].index
missing_lead = np.array(missing_lead)

lead_plus_min = np.append(missing_lead-1, missing_lead+1)
lead_plus_min

In [ ]:
#  replacing the '       .' value with NaN
lead_df['Lead'] = lead_df['Lead'].replace('       .', np.nan, regex=False)

# Typecasting the Year and Lead columns
lead_df['Year'] = lead_df['Year'].str.replace('JJ00','').astype(int)
lead_df['Lead'] = lead_df['Lead'].astype(float) # Float for now because NaN can't be int.

# Filling the NaN with an interpolated value
lead_df['Lead'] = lead_df['Lead'].interpolate().astype(int)

In [ ]:
# Now to check if the interpolated data make sense

measured_lead_scatter = lead_df.iloc[lead_plus_min
                                     ].hvplot.scatter(x='Vol_Wastewater',
                                                      y='Lead',
                                                      label='Experimental data')

interpolated_lead_scatter = lead_df.iloc[missing_lead
                                     ].hvplot.scatter(x='Vol_Wastewater',
                                                      y='Lead', color='red',
                                                      label='Interpolated data')
                                     
factcheck = measured_lead_scatter * interpolated_lead_scatter
factcheck.opts(title='Measuered and interpolated lead data',
               ylabel='Lead (kg)', xlabel='Volume Infulent Wastewater (m3)',
               legend_position='top_left')



Now we have the amount of wastewater in 1000 m3 and the amount of lead in the water in kg, I would like to create a column with the amount of lead per m3 of water

In [ ]:
lead_df['lead_per_m3'] = lead_df['Lead'] / lead_df['Vol_Wastewater']
# Converting lead from kilogram to gram
lead_df['lead_per_m3'] = lead_df['lead_per_m3']*1000
lead_df

In [ ]:
# Lets check the data again now we're done cleaning and interpolating values.
check_data(lead_df)

Now I'm going to have a look at the Region column in both DataFrames, since this is the feature I'll be merging on

In [ ]:
print(f"""
Crime regions:
{crime_df['Provence'].unique()}

Lead regions:
{lead_df['Provence'].unique()}""")

There clearly is some whitespace that needs removing.

In [ ]:
crime_df['Provence'] = crime_df['Provence'].str.replace(r'\s','', regex=True)
lead_df['Provence'] = lead_df['Provence'].str.replace(r'\s','', regex=True)

In [ ]:
print(f"""
Crime regions:
{crime_df['Provence'].unique()}

Lead regions:
{lead_df['Provence'].unique()}""")

Now both dataframes are ready to be merged!

In [ ]:
lead_crime_df = lead_df.merge(crime_df, how='inner', on=['Provence', 'Year'], 
                              copy=True)
lead_crime_df = lead_crime_df.drop(['Vol_Wastewater', 'Lead', 'ID'], axis=1)
lead_crime_df['Provence'] = lead_crime_df['Provence'].map({'PV20': 'Groningen',
                                                           'PV21': 'Fryslân',
                                                           'PV22': 'Drenthe',
                                                           'PV23': 'Overijssel',
                                                           'PV24': 'Flevoland',
                                                           'PV25': 'Gelderland',
                                                           'PV26': 'Utrecht',
                                                           'PV27': 'Noord-Holland',
                                                           'PV28': 'Zuid-Holland',
                                                           'PV29': 'Zeeland',
                                                           'PV30': 'Noord-Brabant',
                                                           'PV31': 'Limburg'})

lead_crime_df.rename(columns={'CRI1110':'Violent theft and burglary',
                              'CRI1500':'Extortion',
                              'CRI2100':'Vandalism',
                              'CRI2210':'Public violence',
                              'CRI2300':'Arson / Explosion',
                              'CRI3000':'Violent an sexual crimes',
                              'CRI7000':'Firearms crimes'},
                     inplace=True)

lead_crime_df

Above is the dataframe I'll be working with to answer my research question. Below I'll provide some information

Columns:
- lead_per_m3: This is the amount of grams of lead found in the incoming waste water
- Incidence: This is the incidence of violent crimes per 100.000 inhabitants of a province

In [ ]:
ds.DS_Q_Q_Plot(crime_df['Total_incidence'])
ds.DS_Q_Q_Hist(crime_df['Total_incidence'])

In [ ]:
ds.DS_Q_Q_Plot(lead_df['lead_per_m3'])
ds.DS_Q_Q_Hist(lead_df['lead_per_m3'])

Looking at the histogram of the total crime incidence values it's clear this data is not normally distributed. I'll take this into account when calculating a correlation between lead and crime

To see how the lead and total indicende behave together I will plot them together on a line plot over time. To account for the difference in y-values I'll first normaize the data. 

In [ ]:
normalized_lead_crime = lead_crime_df[['Provence','Year',
                                       'lead_per_m3','Total_incidence']].copy()

for col in ['lead_per_m3', 'Total_incidence']:
    
    df_max = normalized_lead_crime[col].max()
    df_min = normalized_lead_crime[col].min()
    df_range = df_max - df_min

    normalized_lead_crime[col] = normalized_lead_crime[col
                                    ].apply(lambda x: ((x-df_min)/df_range))

normalized_lead_crime


In [ ]:
normalized_lead_crime.hvplot(kind='line', x='Year', 
                     y=['lead_per_m3','Total_incidence'], 
                     groupby='Provence').opts(legend_position='top_right')

Two observations here:
- In all provinces the incidence of violent crimes decrease over time (hooray).
- The lead content in inffluent waste water in most provencesjumps around over time. Sometime ending a higher, sometimes lower. Not very hopeful for a strong correlation

In [ ]:
Crimes = ['Violent theft and burglary','Extortion', 'Vandalism', 
          'Public violence', 'Arson / Explosion', 'Violent an sexual crimes', 
          'Firearms crimes']

hv.output(widget_location='top_left')
lead_crime_df.set_index('Provence'
                        ).hvplot(kind='bar', stacked=True,
                                 y=Crimes, groupby='Year',
                                 ylim=(0,25), title='Crimes per region, per year.',
                                 ylabel='Incidence per 100.000 inhabitants',
                                 rot=40)


To answer my research question - Is there a correlation between lead in water and violent crimes - I'm going to perform a Pearson Correlation test

In [ ]:
# Onderbouwen regressie en de checks
# Onderstaande per provincie
# Testen voor outliers

In [ ]:
from scipy.stats import spearmanr
correlation, p_val = spearmanr(lead_crime_df['lead_per_m3'], 
                              lead_crime_df['Total_incidence'])
print(f"""
The correlation between the lead per cubic meter of water and
total incidence of violent crimes is {correlation:.2f}, p value: {p_val}.
""")

In [ ]:
scatter_data = hv.Scatter(data=lead_crime_df, kdims='lead_per_m3', 
                          vdims='Total_incidence', label='Regression line')
scatter = lead_crime_df.hvplot.scatter(
                            x='lead_per_m3',
                            y='Total_incidence',
                            xlabel='Lead per m3 of water (in grams)',
                            ylabel='Total incidence per 100.000 inhabitants',
                            label='Data',
                            title='Ja')

Regression_line = hv.Slope.from_scatter(scatter_data).opts(color='black')

scatter.opts(legend_position='top_right') * Regression_line

In [ ]:
# In the above graph we see four datapoints that seem to be a bit out of the ordinary. 
# I'm going to dorp them to see if this has any impact.

to_drop = lead_crime_df[(lead_crime_df['lead_per_m3'] < 6) &
                        (lead_crime_df['Total_incidence'] > 18)
                        ].index.tolist()

to_drop.extend(lead_crime_df[(lead_crime_df['lead_per_m3'] > 35) & 
                             (lead_crime_df['Total_incidence'] < 14)
                             ].index.tolist())
to_drop

lead_crime_df.drop(to_drop, inplace=True)

In [ ]:
correlation, p_val = spearmanr(lead_crime_df['lead_per_m3'], 
                              lead_crime_df['Total_incidence'])
print(f"""
The correlation between the lead per cubic meter of water and
total incidence of violent crimes is {correlation:.2f}, p value: {p_val}.
""")

In [ ]:
# So while this does impact the correlation somewhat, 
# it does not increase it to a significant level.

In [ ]:
scatter_data = hv.Scatter(data=lead_crime_df, kdims='lead_per_m3', 
                          vdims='Total_incidence')

scatter = lead_crime_df.hvplot.scatter(
                            x='lead_per_m3',
                            y='Total_incidence',
                            xlabel='Lead per m3 of water (in grams)',
                            ylabel='Total incidence per 100.000 inhabitants',
                            label='Data',
                            title='Ja')

Regression_line2 = hv.Slope.from_scatter(scatter_data).opts(color='red')

(scatter.opts(legend_position='top_right') * Regression_line
 * Regression_line2)

In [ ]:
# This is confirmed visually.

So, looking at the total incidence and lead per m3 of water, there is no significant correlation. Next I'm going to look at the data per provence to see if there are trends visible on a smaller scale

In [ ]:
corr_lead_crime_province = lead_crime_df[['lead_per_m3','Total_incidence','Provence']
                                        ].groupby('Provence'
                                        ).corr(method='spearman').reset_index(
                                            )[::2][['Provence','Total_incidence']]

corr_lead_crime_province.rename(columns={'Total_incidence':'Correlation lead & crime'}, 
                                inplace=True)

corr_lead_crime_province.set_index('Provence').hvplot.bar(rot=40,
                                                         xlabel='Provence of the Netherlands')

In [ ]:
# Stratify per crime and / or Provence
scatter_data = hv.Scatter(data=lead_crime_df, kdims='lead_per_m3', 
                          vdims='Total_incidence', label='Regression line')

scatter = lead_crime_df.hvplot.scatter(
                            x='lead_per_m3',
                            y='Total_incidence',
                            groupby='Provence',
                            xlabel='Lead per m3 of water (in grams)',
                            ylabel='Total incidence per 100.000 inhabitants',
                            label='Data', title='Ja', subplots=True)

Regression_line2 = hv.Slope.from_scatter(scatter_data).opts(color='red')

scatter.opts(legend_position='top_right') * Regression_line * Regression_line2


Finally, I want to plot the incidence and lead per m3 of wastewater per provence, over a map of the Netherlands.

In [ ]:
# Geoplotten

def get_map():
    m = folium.Map(location=[52.2594406805025, 4.952074743739346],
                   zoom_start=7)
    
    url = 'https://www.webuildinternet.com/articles/2015-07-19-geojson-data-of-the-netherlands/provinces.geojson'

    
    borderstyle = {
        'color' : 'black',
        'fill' : False,
        'weight' : 1
    }
    
    border = folium.GeoJson(data=url, name='borders', 
                             style_function= lambda x: borderstyle, 
                             control = False).add_to(m)

    incidence = folium.Choropleth(
        geo_data=url,
        name="Incidence",
        data=lead_crime_df.replace('Fryslân', 'Friesland (Fryslân)'),
        columns=["Provence", "Total_incidence"],
        key_on="properties.name", # Thanks to job for helping me figure out wich value to use here!
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=1,
        legend_name="Total violent crime incidence (per 100.000 inhabitants)",
        show = False
    ).add_to(m)

    lead = folium.Choropleth(
            geo_data=url,
            name="Lead",
            data=lead_crime_df.replace('Fryslân', 'Friesland (Fryslân)'),
            columns=["Provence", "lead_per_m3"],
            key_on="properties.name",
            fill_color="BuPu",
            fill_opacity=0.7,
            line_opacity=1,
            legend_name="Total violent crime incidence (per 100.000 inhabitants)",
            show = False,
            hilight = True
    ).add_to(m)

    minimap = plugins.MiniMap(toggle_display=True)
    m.add_child(minimap)

    folium.LayerControl(collapsed=False).add_to(m)

    return m

mapobject = get_map()
mapobject.save('Lead & Crime incidence NL.html')

mapobject

Conclusion: There is no significant correlation between lead found in wastewater and violent crimes, anywhere in the Netherlands. Only in the provence Overijssel there is a somewhat strong negative correlation between lead in influent waste water and violent crime but looking at the rest of the data, I suspect this does not indicate any causation. 